## Paramterized Ethane Example

In [2]:
import pytest 
import numpy as np
import mbuild as mb 
import mbuild.recipes 
import unyt as u 
import foyer 
import gmso 

from gmso.external import from_parmed
import parmed as pmd

import warnings 
warnings.filterwarnings('ignore')

/Users/quachcd/opt/anaconda3/envs/general/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
from mbuild.lib.molecules import Ethane

mb_ethane = Ethane()
oplsaa = foyer.Forcefield(name='oplsaa')

pmd_ethane = oplsaa.apply(mb_ethane)
top = from_parmed(pmd_ethane)
top.name = "Ethane"

In [4]:
for site in top.sites:
    print('Atom name: {}'.format(site.name))
    print('Atom type: {} \n'.format(site.atom_type.name))

Atom name: C
Atom type: opls_135 

Atom name: H
Atom type: opls_140 

Atom name: H
Atom type: opls_140 

Atom name: H
Atom type: opls_140 

Atom name: C
Atom type: opls_135 

Atom name: H
Atom type: opls_140 

Atom name: H
Atom type: opls_140 

Atom name: H
Atom type: opls_140 



In [5]:
display(top.sites[0].atom_type.__dict__)

{'name_': 'opls_135',
 'potential_expression_': <PotentialExpression, expression: 4*epsilon*(-sigma**6/r**6 + sigma**12/r**12), 1 independent variables>,
 'topology_': <Ethane 8 sites, 28 connections, id: 140246136100816>,
 'set_ref_': 'atom_type_dict',
 'mass_': unyt_quantity(12.01078, 'g/mol'),
 'charge_': unyt_quantity(0., 'C'),
 'atomclass_': '',
 'doi_': '',
 'overrides_': set(),
 'definition_': '',
 'description_': ''}

In [6]:
potential_object = top.sites[0].atom_type.potential_expression
print('Potential expression: {}'.format(potential_object.expression))
print('Independent variables: {}'.format(potential_object.independent_variables))
print('Paremeters: {}'.format(potential_object.parameters))

Potential expression: 4*epsilon*(-sigma**6/r**6 + sigma**12/r**12)
Independent variables: {r}
Paremeters: {'sigma': unyt_quantity(3.5, 'Å'), 'epsilon': unyt_quantity(0.066, 'kcal/mol')}


## GROMACS and LAMMPS writer

In [1]:
from gmso.formats.gro import read_gro, write_gro
from gmso.formats.top import write_top
from gmso.formats.lammpsdata import write_lammpsdata, read_lammpsdata

In [8]:
write_gro(top, './simulations/gromacs/out.gro')
write_top(top, './simulations/gromacs/out.top')
#write_lammpsdata(top, './simulations/lammps/ethane.lammps')

In [16]:
%cd ./simulations/gromacs/
!gmx grompp -v -f em.mdp -p out.top -c out.gro -o em.tpr 
!gmx mdrun -v -deffnm em -s em.tprt


/Users/quachcd/Documents/dev/gmso_showcase/ethane_example/simulations/gromacs
                      :-) GROMACS - gmx grompp, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf  

In [20]:
top.box.lengths

unyt_array([7.13999987, 7.93800011, 6.646     ], 'Å')